In [ ]:
#1. Create a Flask app that displays "Hello, World!" on the homepage.

In [ ]:
pip install flask

In [ ]:
from flask import Flask
import threading

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, World!"

def run_app():
    app.run(host='localhost', port=5000)

thread = threading.Thread(target=run_app)
thread.start()

In [ ]:
#2. Build a Flask app with static HTML pages and navigate between them.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
#3. Develop a Flask app that uses URL parameters to display dynamic content.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/user/<name>')
def user(name):
    return render_template('user.html', name=name)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
#4. Create a Flask app with a form that accepts user input and displays it.

In [ ]:
@app.route('/form', methods=['GET', 'POST'])
def form():
    if request.method == 'POST':
        user_input = request.form['user_input']
        return render_template('result.html', user_input=user_input)
    return render_template('form.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
#5. Implement user sessions in a Flask app to store and display user-specific data.

In [ ]:
from flask import Flask, render_template, request, session

app = Flask(__name__)
app.secret_key = 'your_secret_key'

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/user/<name>')
def user(name):
    session['name'] = name
    return render_template('user.html', name=name)

@app.route('/form', methods=['GET', 'POST'])
def form():
    if request.method == 'POST':
        user_input = request.form['user_input']
        session['user_input'] = user_input
        return render_template('result.html', user_input=user_input)
    return render_template('form.html')

@app.route('/session')
def session_data():
    name = session.get('name', 'Guest')
    user_input = session.get('user_input', 'No input yet')
    return f'User: {name}, Last Input: {user_input}'

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
#6. Build a Flask app that allows users to upload files and display them on the website.

In [ ]:
from flask import Flask, render_template, request, session, redirect, url_for
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config['UPLOAD_FOLDER'] = 'uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/user/<name>')
def user(name):
    session['name'] = name
    return render_template('user.html', name=name)

@app.route('/form', methods=['GET', 'POST'])
def form():
    if request.method == 'POST':
        user_input = request.form['user_input']
        session['user_input'] = user_input
        return render_template('result.html', user_input=user_input)
    return render_template('form.html')

@app.route('/session')
def session_data():
    name = session.get('name', 'Guest')
    user_input = session.get('user_input', 'No input yet')
    return f'User: {name}, Last Input: {user_input}'

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return f'File {filename} uploaded successfully!'
    return render_template('upload.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
#7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

In [ ]:
from flask import Flask, render_template, request, session, redirect, url_for, g
import os
import sqlite3
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config['UPLOAD_FOLDER'] = 'uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

DATABASE = 'database.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
        db.execute("CREATE TABLE IF NOT EXISTS items (id INTEGER PRIMARY KEY, name TEXT)")
    return db

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

@app.route('/')
def home():
    db = get_db()
    cur = db.cursor()
    cur.execute("SELECT * FROM items")
    items = cur.fetchall()
    return render_template('home.html', items=items)

@app.route('/add', methods=['POST'])
def add_item():
    name = request.form['name']
    db = get_db()
    db.execute("INSERT INTO items (name) VALUES (?)", (name,))
    db.commit()
    return redirect(url_for('home'))

@app.route('/delete/<int:item_id>')
def delete_item(item_id):
    db = get_db()
    db.execute("DELETE FROM items WHERE id = ?", (item_id,))
    db.commit()
    return redirect(url_for('home'))

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/user/<name>')
def user(name):
    session['name'] = name
    return render_template('user.html', name=name)

@app.route('/form', methods=['GET', 'POST'])
def form():
    if request.method == 'POST':
        user_input = request.form['user_input']
        session['user_input'] = user_input
        return render_template('result.html', user_input=user_input)
    return render_template('form.html')

@app.route('/session')
def session_data():
    name = session.get('name', 'Guest')
    user_input = session.get('user_input', 'No input yet')
    return f'User: {name}, Last Input: {user_input}'

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return f'File {filename} uploaded successfully!'
    return render_template('upload.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
#8. Implement user authentication and registration in a Flask app using Flask-Login.

In [ ]:
from flask import Flask, render_template, request, session, redirect, url_for, g
import os
import sqlite3
from werkzeug.utils import secure_filename
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config['UPLOAD_FOLDER'] = 'uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

DATABASE = 'database.db'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

class User(UserMixin):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
        db.execute("CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, username TEXT UNIQUE, password TEXT)")
        db.execute("CREATE TABLE IF NOT EXISTS items (id INTEGER PRIMARY KEY, name TEXT)")
    return db

@login_manager.user_loader
def load_user(user_id):
    db = get_db()
    cur = db.cursor()
    cur.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    user = cur.fetchone()
    if user:
        return User(user[0], user[1], user[2])
    return None

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

@app.route('/')
def home():
    db = get_db()
    cur = db.cursor()
    cur.execute("SELECT * FROM items")
    items = cur.fetchall()
    return render_template('home.html', items=items, user=current_user)

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        db = get_db()
        try:
            db.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
            db.commit()
            return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            return "Username already exists!"
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        db = get_db()
        cur = db.cursor()
        cur.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, password))
        user = cur.fetchone()
        if user:
            login_user(User(user[0], user[1], user[2]))
            return redirect(url_for('home'))
        return "Invalid credentials!"
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('home'))

@app.route('/add', methods=['POST'])
@login_required
def add_item():
    name = request.form['name']
    db = get_db()
    db.execute("INSERT INTO items (name) VALUES (?)", (name,))
    db.commit()
    return redirect(url_for('home'))

@app.route('/delete/<int:item_id>')
@login_required
def delete_item(item_id):
    db = get_db()
    db.execute("DELETE FROM items WHERE id = ?", (item_id,))
    db.commit()
    return redirect(url_for('home'))

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

In [ ]:
#9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

In [ ]:
from flask import Flask, request, jsonify, g
import os
import sqlite3
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)
app.secret_key = 'your_secret_key'
DATABASE = 'database.db'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

class User(UserMixin):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
        db.execute("CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY, username TEXT UNIQUE, password TEXT)")
        db.execute("CREATE TABLE IF NOT EXISTS books (id INTEGER PRIMARY KEY, title TEXT, author TEXT)")
    return db

@login_manager.user_loader
def load_user(user_id):
    db = get_db()
    cur = db.cursor()
    cur.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    user = cur.fetchone()
    if user:
        return User(user[0], user[1], user[2])
    return None

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

@app.route('/books', methods=['GET'])
def get_books():
    db = get_db()
    cur = db.cursor()
    cur.execute("SELECT * FROM books")
    books = [{'id': row[0], 'title': row[1], 'author': row[2]} for row in cur.fetchall()]
    return jsonify(books)

@app.route('/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    db = get_db()
    cur = db.cursor()
    cur.execute("SELECT * FROM books WHERE id = ?", (book_id,))
    book = cur.fetchone()
    if book:
        return jsonify({'id': book[0], 'title': book[1], 'author': book[2]})
    return jsonify({'error': 'Book not found'}), 404

@app.route('/books', methods=['POST'])
@login_required
def add_book():
    data = request.get_json()
    title = data.get('title')
    author = data.get('author')
    db = get_db()
    cur = db.cursor()
    cur.execute("INSERT INTO books (title, author) VALUES (?, ?)", (title, author))
    db.commit()
    return jsonify({'message': 'Book added successfully'}), 201

@app.route('/books/<int:book_id>', methods=['PUT'])
@login_required
def update_book(book_id):
    data = request.get_json()
    title = data.get('title')
    author = data.get('author')
    db = get_db()
    cur = db.cursor()
    cur.execute("UPDATE books SET title = ?, author = ? WHERE id = ?", (title, author, book_id))
    db.commit()
    return jsonify({'message': 'Book updated successfully'})

@app.route('/books/<int:book_id>', methods=['DELETE'])
@login_required
def delete_book(book_id):
    db = get_db()
    cur = db.cursor()
    cur.execute("DELETE FROM books WHERE id = ?", (book_id,))
    db.commit()
    return jsonify({'message': 'Book deleted successfully'})

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)


In [ ]:
# 10. Design a Flask app with proper error handling for 404 and 500 errors.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

# Define a simple route
@app.route('/')
def home():
    return 'Welcome to the Flask App!'

# Custom 404 Error handler
@app.errorhandler(404)
def page_not_found(error):
    return render_template('404.html'), 404

# Custom 500 Error handler
@app.errorhandler(500)
def internal_server_error(error):
    return render_template('500.html'), 500

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
#11. Create a real-time chat application using Flask-SocketIO.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, send

app = Flask(__name__)
app.secret_key = "secret!"  # You can use a proper secret key in production
socketio = SocketIO(app)

# Route to render the chat page
@app.route('/')
def index():
    return render_template('index.html')

# Event to handle sending messages
@socketio.on('message')
def handle_message(msg):
    print(f"Received message: {msg}")
    send(msg, broadcast=True)  # Broadcast the message to all connected clients

if __name__ == '__main__':
    socketio.run(app, debug=True)

In [ ]:
#12. Build a Flask app that updates data in real-time using WebSocket connections.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import random
import time
from threading import Thread

app = Flask(__name__)
app.secret_key = "secret!"
socketio = SocketIO(app)

# Function to generate and emit random data every 2 seconds
def generate_data():
    while True:
        # Simulating data generation (e.g., a random number)
        data = random.randint(1, 100)
        # Emit the data to all connected clients
        socketio.emit('update_data', {'data': data}, broadcast=True)
        time.sleep(2)

# Start the data generation in a separate thread
@app.before_first_request
def start_data_generation():
    thread = Thread(target=generate_data)
    thread.daemon = True
    thread.start()

# Route to render the main page
@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    socketio.run(app, debug=True)

In [ ]:
#13. Implement notifications in a Flask app using websockets to notify users of updates.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import time
from threading import Thread

app = Flask(__name__)
app.secret_key = "secret!"
socketio = SocketIO(app)

# Function to simulate event and send notifications
def send_notifications():
    while True:
        time.sleep(5)  # Simulate an event every 5 seconds
        notification = "New update available!"  # Notification message
        # Send the notification to all connected clients
        socketio.emit('new_notification', {'message': notification}, broadcast=True)

# Start the notification thread when the first request comes
@app.before_first_request
def start_notification_thread():
    thread = Thread(target=send_notifications)
    thread.daemon = True
    thread.start()

# Route to render the home page
@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    socketio.run(app, debug=True)